In [104]:
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys
import time
import warnings
import matplotlib as mpl

import tensorflow as tf
from tensorflow import keras


torch.cuda.is_available()

True

In [105]:
x = torch.rand(5, 3)
print(x)

tensor([[0.6890, 0.0693, 0.6618],
        [0.4890, 0.5596, 0.4511],
        [0.6423, 0.7833, 0.1593],
        [0.7911, 0.4824, 0.7778],
        [0.6661, 0.2106, 0.8153]])


In [106]:

import os.path


train_all = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train_all = train_all.dropna()

NUMERIC_FEATURE_NAMES = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
]

CATEGORICAL_FEATURE_NAMES = ["HomePlanet","Destination"]


train_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6606 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6606 non-null   object 
 1   HomePlanet    6606 non-null   object 
 2   CryoSleep     6606 non-null   object 
 3   Cabin         6606 non-null   object 
 4   Destination   6606 non-null   object 
 5   Age           6606 non-null   float64
 6   VIP           6606 non-null   object 
 7   RoomService   6606 non-null   float64
 8   FoodCourt     6606 non-null   float64
 9   ShoppingMall  6606 non-null   float64
 10  Spa           6606 non-null   float64
 11  VRDeck        6606 non-null   float64
 12  Name          6606 non-null   object 
 13  Transported   6606 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 729.0+ KB


In [107]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [108]:
test_data_file = "test_data.csv"
test.to_csv(test_data_file, index=False, header=True)

train_all["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
train_all["VIP"].replace([True,False], ["1", "0"], inplace=True)
test["CryoSleep"].replace([True,False], ["1", "0"], inplace=True)
test["VIP"].replace([True,False], ["1", "0"], inplace=True)

train_all["Relation"] = train_all["PassengerId"].str.split("_").str[1].astype(float)
test["Relation"] = test["PassengerId"].str.split("_").str[1].astype(float)

# train_all["Deck"] = train_all["Cabin"].str[0]
# test["Deck"] = test["Cabin"].str[0]

# train_all["Side"] = train_all["Cabin"].str[2]
# test["Side"] = test["Cabin"].str[2]


train_all.drop(['PassengerId', 'Cabin','Name'],axis =1 , inplace=True)
test.drop(['PassengerId', 'Cabin','Name'],axis = 1 ,inplace=True)
train_all.Transported.replace([True,False], ["1", "0"], inplace=True)

In [109]:


# train_all["Age"].fillna(train_all["Age"].mean(skipna=True), inplace=True)
test["Age"].fillna(test["Age"].mean(skipna=True), inplace=True)

# train_all["Age"] = train_all["Age"] / train_all["Age"].max()
# test["Age"] = test["Age"] / test["Age"].max()

# train_all["RoomService"].fillna(train_all["RoomService"].mean(skipna=True), inplace=True)
test["RoomService"].fillna(test["RoomService"].mean(skipna=True), inplace=True)

# train_all["FoodCourt"].fillna(train_all["FoodCourt"].mean(skipna=True), inplace=True)
test["FoodCourt"].fillna(test["FoodCourt"].mean(skipna=True), inplace=True)

# train_all["ShoppingMall"].fillna(train_all["ShoppingMall"].mean(skipna=True), inplace=True)
test["ShoppingMall"].fillna(test["ShoppingMall"].mean(skipna=True), inplace=True)

# train_all["Spa"].fillna(train_all["Spa"].mean(skipna=True), inplace=True)
test["Spa"].fillna(test["Spa"].mean(skipna=True), inplace=True)

# train_all["VRDeck"].fillna(train_all["VRDeck"].mean(skipna=True), inplace=True)
test["VRDeck"].fillna(test["VRDeck"].mean(skipna=True), inplace=True)

#  ["HomePlanet","CryoSleep","Destination","VIP"]

# train_all["HomePlanet"].fillna('Kepler', inplace=True)
test["HomePlanet"].fillna('Earth', inplace=True)

# train_all["CryoSleep"].fillna('0', inplace=True)
test["CryoSleep"].fillna('0', inplace=True)

# train_all["VIP"].fillna('0', inplace=True)
test["VIP"].fillna('0', inplace=True)

# train_all["Destination"].fillna('TRAPPIST-1e', inplace=True)
test["Destination"].fillna('TRAPPIST-1e', inplace=True)


# test.dropna(inplace = True)

In [110]:
planets = train_all["HomePlanet"].unique()
indices = tf.range(len(planets), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(planets, indices)

table = tf.lookup.StaticVocabularyTable(table_init, 1)

In [111]:
aaa = tf.constant(train_all["HomePlanet"].values)
cat_indices = table.lookup(aaa)
cat_one_hot = tf.one_hot(cat_indices, 5)

cat_one_hot


<tf.Tensor: shape=(6606, 5), dtype=float32, numpy=
array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)>

In [112]:

train_all = pd.get_dummies(train_all, columns=["HomePlanet","Destination"])
test = pd.get_dummies(test, columns=["HomePlanet","Destination"])

# for planet in planets:
#   train_all[planet].loc[train_all["HomePlanet"] == planet] = 1
#   train_all[planet].loc[train_all["HomePlanet"] != planet] = 0

train_all.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Relation,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0,0,1,0,0,0,1
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,1.0,1,0,0,0,0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,1.0,0,1,0,0,0,1
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,2.0,0,1,0,0,0,1
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1.0,1,0,0,0,0,1


In [113]:
test.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Relation,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1,27.0,0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0,0,1
1,0,19.0,0,0.0,9.0,0.0,2823.0,0.0,1.0,1,0,0,0,0,1
2,1,31.0,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,1,0,0
3,0,38.0,0,0.0,6652.0,0.0,181.0,585.0,1.0,0,1,0,0,0,1
4,0,20.0,0,10.0,0.0,635.0,0.0,0.0,1.0,1,0,0,0,0,1


In [114]:
# CSV_HEADER = [
#     "HomePlanet",
#     "Destination",
#     "Age",
#     "RoomService",
#     "FoodCourt",
#     "ShoppingMall",
#     "Spa",
#     "VRDeck",
#     "Transported",
#     ]

train_data_file = "train_data.csv"


In [115]:
# def get_dataset_from_csv(csv_file_path, shuffle=False, batch_size=128):
#     dataset = tf.data.experimental.make_csv_dataset(
#         csv_file_path,
#         batch_size=batch_size,
#         column_names=CSV_HEADER,
#         label_name="Transported",
#         num_epochs=1,
#         shuffle=shuffle
#     )
#     return dataset.cache()

In [116]:
# train_all = pd.read_csv('./train_data.csv')
# test = pd.read_csv('./test_data.csv')

train, valid = train_test_split(train_all, test_size=0.05, random_state=42, shuffle=True)

train_X, train_y = train.drop('Transported', axis=1), train['Transported']
valid_X, valid_y = valid.drop('Transported', axis=1), valid['Transported']

train_X.to_csv("./train_x.csv", index=False, header=False)
train_y.to_csv("./train_y.csv", index=False, header=False)
train_X.to_csv("./valid_x.csv", index=False, header=False)
train_y.to_csv("./valid_y.csv", index=False, header=False)

test.to_csv("./test_v.csv", index=False, header=False)


train_X, train_y = pd.read_csv('./train_x.csv'), pd.read_csv('./train_y.csv')
valid_X, valid_y = pd.read_csv('./valid_x.csv'), pd.read_csv('./valid_y.csv')

test_v = pd.read_csv('./test_v.csv')



train_X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6274 entries, 0 to 6273
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       6274 non-null   int64  
 1   30.0    6274 non-null   float64
 2   0.1     6274 non-null   int64  
 3   0.0     6274 non-null   float64
 4   0.0.1   6274 non-null   float64
 5   0.0.2   6274 non-null   float64
 6   27.0    6274 non-null   float64
 7   764.0   6274 non-null   float64
 8   1.0     6274 non-null   float64
 9   1       6274 non-null   int64  
 10  0.2     6274 non-null   int64  
 11  0.3     6274 non-null   int64  
 12  0.4     6274 non-null   int64  
 13  0.5     6274 non-null   int64  
 14  1.1     6274 non-null   int64  
dtypes: float64(7), int64(8)
memory usage: 735.4 KB


In [117]:
test_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4276 entries, 0 to 4275
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       4276 non-null   int64  
 1   27.0    4276 non-null   float64
 2   0       4276 non-null   int64  
 3   0.0     4276 non-null   float64
 4   0.0.1   4276 non-null   float64
 5   0.0.2   4276 non-null   float64
 6   0.0.3   4276 non-null   float64
 7   0.0.4   4276 non-null   float64
 8   1.0     4276 non-null   float64
 9   1.1     4276 non-null   int64  
 10  0.1     4276 non-null   int64  
 11  0.2     4276 non-null   int64  
 12  0.3     4276 non-null   int64  
 13  0.4     4276 non-null   int64  
 14  1.2     4276 non-null   int64  
dtypes: float64(7), int64(8)
memory usage: 501.2 KB


In [118]:

# train_X["relation"] = train_X["PassengerId"].str.split("_").str[1].astype(int)
# train_X["relation"]

In [119]:
# train_X = np.asarray(train_X).astype('float32')


In [120]:
test = np.asarray(test).astype('float32')
test

array([[ 1.      , 27.      ,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 0.      , 19.      ,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 1.      , 31.      ,  0.      , ...,  1.      ,  0.      ,
         0.      ],
       ...,
       [ 1.      , 28.658146,  0.      , ...,  1.      ,  0.      ,
         0.      ],
       [ 0.      , 28.658146,  0.      , ...,  0.      ,  0.      ,
         1.      ],
       [ 1.      , 43.      ,  0.      , ...,  0.      ,  1.      ,
         0.      ]], dtype=float32)

In [121]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.Sequential([
    keras.layers.Dense(16, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dense(128, activation='elu'),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['binary_accuracy'])

In [122]:
model.fit(train_X, train_y, epochs=20, validation_data=(valid_X, valid_y), batch_size=150)

Epoch 1/20
42/42 [==============================] - 1s 12ms/step - loss: 0.5557 - binary_accuracy: 0.7381 - val_loss: 0.5230 - val_binary_accuracy: 0.7973
Epoch 2/20
42/42 [==============================] - 0s 8ms/step - loss: 0.5091 - binary_accuracy: 0.7729 - val_loss: 0.5279 - val_binary_accuracy: 0.7906
Epoch 3/20
42/42 [==============================] - 0s 7ms/step - loss: 0.4986 - binary_accuracy: 0.7815 - val_loss: 0.4962 - val_binary_accuracy: 0.7829
Epoch 4/20
42/42 [==============================] - 0s 7ms/step - loss: 0.4962 - binary_accuracy: 0.7839 - val_loss: 0.4983 - val_binary_accuracy: 0.7909
Epoch 5/20
42/42 [==============================] - 0s 7ms/step - loss: 0.4953 - binary_accuracy: 0.7864 - val_loss: 0.4858 - val_binary_accuracy: 0.7914
Epoch 6/20
42/42 [==============================] - 0s 8ms/step - loss: 0.4863 - binary_accuracy: 0.7894 - val_loss: 0.4813 - val_binary_accuracy: 0.7891
Epoch 7/20
42/42 [==============================] - 0s 7ms/step - loss: 0.4

In [123]:
y_pred = model.predict(test)
y_pred

array([[0.70997083],
       [0.01642066],
       [0.7479648 ],
       ...,
       [0.7369322 ],
       [0.6940927 ],
       [0.7661327 ]], dtype=float32)

In [124]:
prediction = pd.DataFrame(y_pred, columns=['Transported'])

prediction["PassengerId"] = pd.read_csv('./test_data.csv')["PassengerId"]

prediction["Transported"].loc[prediction['Transported'] > 0.5] = True
prediction["Transported"].loc[prediction['Transported'] <= 0.5] = False

prediction.to_csv("./submit.csv", index=False, header=True)


# .to_csv('submit.csv')
# prediction

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
